In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import pandas as pd
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [3]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_learning_New_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_learning_dict_New_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense

In [4]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [6]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [7]:
df = pd.read_csv("../dataset/2020산림복지_본문_맞춤법검사.txt", encoding='UTF-8')
df

,Unnamed: 0,split_str,org_idx,correct_str
0,0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...
1,1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...
2,2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...
3,3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...
4,4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...
...,...,...,...,...
164988,164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.
164989,164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다
164990,164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...
164991,164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.


In [8]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

100%|██████████| 164993/164993 [2:31:29<00:00, 18.15it/s]  


In [9]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,부정,0
1,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,긍정,0
2,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,긍정,0
3,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,긍정,0
4,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,긍정,0
...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,긍정,5787
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,긍정,5787
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,긍정,5787
164991,2020년 1월 7일 하루 종일 비가 내린다.,긍정,5787


In [10]:
ddff['label'].value_counts()

긍정    136568
부정     28301
중립       124
Name: label, dtype: int64

In [19]:
ddff[ddff['label'] == '중립']

,correct_str,label
30,실제로 교육을 가장 잘 받고 소득도 높고 환경 의식도 투철한 부류가 자원을 가장 많...,중립
31,이렇듯 녹색 지향적인 쾌락주의자는 실용적이고 시장에 바탕을 두고 있으며 기술로 세계...,중립
41,처음에는 채식이 한 개인의 건강을 위해서만 하는 것으로 생각했는데 환경에도 큰 영향...,중립
48,고기는 토착민의 피로부터 나와요.,중립
56,우리는 자연과 함께 전통에 따라 살고 있어요.,중립
...,...,...
164935,또 북구 기초의원 2명도 이번 재보선에서 새로 뽑는다,중립
164940,또 상주시장에도 10명이 넘는 인사가 예비후보로 나섰다.,중립
164942,민주당 무소속 후보도 각각 1명씩 예비후보로 등록했다.,중립
164946,또 안동에서 광역의원 1명 울진에서 기초의원 1명을 뽑는다,중립


In [11]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2020.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
